# TESTE de TF-IDF Manual

## Importações

In [211]:
# Importações
import nltk
import pandas as pd
import re
from collections import Counter
from itertools import chain
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')  # Não baixa se já estiver atualizado!
from nltk.corpus import stopwords
import math

[nltk_data] Downloading package stopwords to /home/vortex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Setup

In [212]:
doc_mestre_path = "datasets/teste_csv_exemplo.csv"
sw = set(stopwords.words('portuguese'))
col_texto = "Texto"
df_mestre = pd.read_csv(doc_mestre_path)

# Funções de auxílio

In [213]:
def ciclar_v(v, n=-1):
    """
    Percorre ciclicamente o vetor v em n passos
    Se n não for fornecido, percorre quantas vezes for chamado
    """
    x = 0
    for _ in range(n):
        yield v[x]
        x = (x + 1) if (x + 1) < len(v) else 0


In [214]:
def aplicar_op(v1, op, v2):
    """
    Aplica operação op nos elementos de v1 a partir dos de v2 e retorna resultado
    em um vetor do mesmo tamanho de v1
    """
    vr = []
    genv2 = ciclar_v(v2, len(v1))
    for i in range(len(v1)):
        vr.append(op(v1[i], next(genv2)))
    return vr

In [215]:
_mul_ = lambda n1, n2: n1 * n2 

## Limpeza

In [216]:
def limpeza_str(texto: str):
    global sw
    texto = texto.lower()
    temp_texto = []
    # Retira todo e qualquer caractere especial (incluindo UNICODE)
    pals = re.sub(r'[^\w\s]|_', ' ', texto, flags=re.UNICODE).split()
    for pal in pals:
        if pal not in sw:
            if pal.isalnum():
                temp_texto.append(pal)
            else:
                temp_texto.append(" ")
    return ' '.join(temp_texto)

In [217]:
df_tinindo = df_mestre.copy()
df_tinindo[col_texto] = df_tinindo[col_texto].apply(limpeza_str)

In [218]:
# query = "A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home."
# query = 'this is the first document'
query = 'O gato comeu o rato'
query = limpeza_str(query)

In [219]:
lista_tinindo = [query] + df_tinindo[col_texto].to_list()
lista_tinindo

['gato comeu rato', 'gato sentou cadeira', 'rato comeu queijo cadeira']

## Funções para cálculo da similaridade de cossenos

$\vec{v}\cdot\vec{w} = \sum_{i = 1}^{n}{\lVert v_{i} \rVert\times\lVert w_{i} \rVert}$, onde $n$ é o número de dimensões

In [220]:
def prod_escalar(v1, v2) -> int:
    """
    Retorna o produto escalar de dois vetores.
    Implicitio que eles tem a mesma dimensão
    """
    return sum(aplicar_op(v1, _mul_, v2))

Fórmula utilizada: $\cos\theta = \frac{\vec{v} \cdot \vec{w}}{\lVert v \rVert \times \lVert w \rVert}$


In [221]:
def comp_sim_cos(list_v, vect) -> list:
    """
    Computa similaridade de cosseno entre uma coleção de vetores e um vetor
    Retorna lista de cossenos
    """
    return [(prod_escalar(v_el, vect)/(math.sqrt(prod_escalar(v_el, v_el))*math.sqrt(prod_escalar(vect, vect)))) for v_el in list_v]

## BOW

## Set com termos únicos (Tokenização)

In [222]:
def dimensionar(lista_limpa) -> tuple:
    """
    Faz, a partir de uma coleção limpa, uma Bag of Words, 
    retornando uma tupla com as dimensões TODO: Decidir se vai ou não ser tupla!
    """
    dimen_set = set()
    for texto in lista_tinindo:
        for pal in texto.split(" "):
            dimen_set.add(pal)
    return tuple(dimen_set)

## Vetorização

In [223]:
lista_dimen = tuple(dimensionar(lista_tinindo))

Passa para tupla para se ter referência fixa dos termos do Set

In [224]:
def arr_bowrizar(lista_limpa):
    arr_dimen = []
    for i, doc in enumerate(lista_limpa):
        vect = []
        for token in lista_dimen:
            vect.append(doc.count(token))
        arr_dimen.append(vect)
    return arr_dimen

In [225]:
col_garcia = ["gato", "comeu", "sentou", "cadeira", "queijo", "rato"]
lista_dimen = col_garcia
arr_dimen = arr_bowrizar(lista_tinindo)
arr_dimen

[[1, 1, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0], [0, 1, 0, 1, 1, 1]]

Dicionários seriam mais práticos, mas também bem mais lentos!

In [226]:
df_bow = pd.DataFrame(arr_dimen, columns=lista_dimen, index=lista_tinindo)
df_bow

,gato,comeu,sentou,cadeira,queijo,rato
gato comeu rato,1,1,0,0,0,1
gato sentou cadeira,1,0,1,1,0,0
rato comeu queijo cadeira,0,1,0,1,1,1


### Similaridade de cossenos por BOW simples

In [227]:
l_sim_cos = comp_sim_cos(arr_dimen, arr_dimen[0])
l_sim_cos

[1.0000000000000002, 0.33333333333333337, 0.5773502691896258]

#### Similaridade por Sci-kit

In [228]:
cvect = CountVectorizer()
f_l = cvect.fit_transform(lista_tinindo)
df_bow_cv = pd.DataFrame(f_l.toarray(), columns=cvect.get_feature_names_out(), index=lista_tinindo)
df_bow_cv = df_bow_cv[lista_dimen]
df_bow_cv


,gato,comeu,sentou,cadeira,queijo,rato
gato comeu rato,1,1,0,0,0,1
gato sentou cadeira,1,0,1,1,0,0
rato comeu queijo cadeira,0,1,0,1,1,1


#### Similaridade de cossenos pelo Sci-kit

In [229]:
sim_scores = cosine_similarity(f_l, f_l[0])
sim_scores.tolist()

[[1.0000000000000002], [0.3333333333333334], [0.5773502691896258]]

## TF (Term Frequency)

In [230]:
def tf(l_vect, rel=True) -> list[list]:
    """
    rel: Define se ocorrerá divisão de cada elemento da coleção por algum valor, relativando
    """
    if rel:
        dimen = len(l_vect[0])
        n_vect = [aplicar_op(vect, _mul_, [1/dimen]) for vect in l_vect]
    else:
        n_vect = l_vect
    return n_vect

In [231]:
l_tf_rel = tf(arr_dimen)
l_tf_rel

[[0.16666666666666666,
  0.16666666666666666,
  0.0,
  0.0,
  0.0,
  0.16666666666666666],
 [0.16666666666666666,
  0.0,
  0.16666666666666666,
  0.16666666666666666,
  0.0,
  0.0],
 [0.0,
  0.16666666666666666,
  0.0,
  0.16666666666666666,
  0.16666666666666666,
  0.16666666666666666]]

In [232]:
l_tf = tf(arr_dimen, False)
l_tf

[[1, 1, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0], [0, 1, 0, 1, 1, 1]]

## IDF (Inverse Documento Frequency)

### Modo manual

In [233]:
# TODO: Esse 'suav' é adequado?
def idf(l_vect, suav=True, modo_garcia=False):
    """
    Coleção de documentos vetorizados -> lista com idfs por termo
    """
    l_idfs = []
    n_docs = len(l_vect)
    n_dimen = len(l_vect[0])  # Pega dimensão do primeiro vetor
    base = 10 if modo_garcia else math.e
    suav = float(suav)
    for i_termo in range(n_dimen):
        # Conta ocorrência transdocumental
        idf_t = math.log(
                (n_docs + suav)/([doc[i_termo] != 0 for doc in l_vect].count(True) + suav),
                base
            ) + float(not modo_garcia)
        l_idfs.append(idf_t)

    return l_idfs
    

In [234]:
idf(arr_dimen, modo_garcia=True)

[0.1249387366082999,
 0.1249387366082999,
 0.30102999566398114,
 0.1249387366082999,
 0.30102999566398114,
 0.1249387366082999]

In [235]:
l_idf = idf(arr_dimen, suav=False, modo_garcia=False)
s_idf = pd.Series(l_idf, index=lista_dimen)
s_idf

gato       1.405465
comeu      1.405465
sentou     2.098612
cadeira    1.405465
queijo     2.098612
rato       1.405465
dtype: float64

### NLTK

In [236]:
vectz = TfidfVectorizer(smooth_idf=False)
tfidf_ = vectz.fit_transform(lista_tinindo)
s_idf = pd.Series(vectz.idf_, index=vectz.get_feature_names_out())
s_idf

cadeira    1.405465
comeu      1.405465
gato       1.405465
queijo     2.098612
rato       1.405465
sentou     2.098612
dtype: float64

## TF-IDF

### Modo manual

In [237]:
def tfidf(*args, modo_garcia=False):
    """
    Aceita lista com strings limpas e bool modo_garcia
    ou TF, IDF, e bool modo_garcia nesta ordem
    """
    if len(args) == 1:
        docs_limpos = args[0]
        arr_bow_ = arr_bowrizar(docs_limpos)
        tf_ = tf(arr_bow_)
        idf_ = idf(arr_bow_, modo_garcia)
    if len(args) == 2:
        tf_ = args[0]
        idf_ = args[1]
    l_tfidf_ = []
    for v in tf_:
        l_tfidf_.append(aplicar_op(v, _mul_, idf_))
    return l_tfidf_

In [238]:
aplicar_op(l_tf[0], _mul_, l_idf)
l_idf

[1.4054651081081644,
 1.4054651081081644,
 2.09861228866811,
 1.4054651081081644,
 2.09861228866811,
 1.4054651081081644]

In [239]:
l_tfidf = tfidf(l_tf, l_idf)
pd_tfidf = pd.DataFrame(l_tfidf, columns=lista_dimen, index=lista_tinindo)
pd_tfidf = pd_tfidf[lista_dimen]
pd_tfidf

,gato,comeu,sentou,cadeira,queijo,rato
gato comeu rato,1.405465,1.405465,0.000000,0.000000,0.000000,1.405465
gato sentou cadeira,1.405465,0.000000,2.098612,1.405465,0.000000,0.000000
rato comeu queijo cadeira,0.000000,1.405465,0.000000,1.405465,2.098612,1.405465


In [240]:
tfidf(lista_tinindo)

[[0.23424418468469405,
  0.23424418468469405,
  0.0,
  0.0,
  0.0,
  0.23424418468469405],
 [0.23424418468469405, 0.0, 0.3497687147780183, 0.23424418468469405, 0.0, 0.0],
 [0.0,
  0.23424418468469405,
  0.0,
  0.23424418468469405,
  0.3497687147780183,
  0.23424418468469405]]

### Pelo Scikit

Para o cálculo do TFIDF o Sci-kit aparenta por padrão utilizar: $tf(t, d)=f_{t,d}$, ou seja, *raw count* de termos naquele documento (BOW)

Já o cálculo do TF-IDF em si é somente: $tfidf(t, d) = f(t,d)\times idf(t, d)$

In [241]:
vectz = TfidfVectorizer(smooth_idf=False, norm=None)
tfidf_ = vectz.fit_transform(lista_tinindo)
pd_tfidf = pd.DataFrame(tfidf_.toarray(), columns=vectz.get_feature_names_out(), index=lista_tinindo)
pd_tfidf = pd_tfidf[lista_dimen]
pd_tfidf

,gato,comeu,sentou,cadeira,queijo,rato
gato comeu rato,1.405465,1.405465,0.000000,0.000000,0.000000,1.405465
gato sentou cadeira,1.405465,0.000000,2.098612,1.405465,0.000000,0.000000
rato comeu queijo cadeira,0.000000,1.405465,0.000000,1.405465,2.098612,1.405465


# Similaridade de cosseno com TF-IDF

## Manual

In [242]:
sim_scores = comp_sim_cos(l_tfidf, l_tfidf[0])
sim_scores

[1.0, 0.280731035410019, 0.5049352628627182]

## Sci-kit

In [243]:
sim_scores = cosine_similarity(tfidf_, tfidf_[0])
sim_scores

array([[1.        ],
       [0.28073104],
       [0.50493526]])

In [ ]:
class Real:
    def __init__(self, valor_str: str):
        list_val = valor_str.split(',', maxsplit=1)
        self.inteiro = int(list_val[0])
        self.decimal = int(list_val[1])
    def __add__(self, other):
        somado = Real()
        somado.intiero = self.inteiro + other.inteiro
        somado.decimal = self.inteiro + other.inteiro
